Generate aligned NIIs with Elastix
We need to align T1c_ax and T2_ax with T1_ax

In [11]:
import itk
from itkwidgets import compare, checkerboard, view
import os
import tqdm
import shutil

in_dir = 'E:\\Parotid\\manual'
out_dir = 'E:\\Parotid\\nii_anon_aligned'

In [2]:
parameter_object = itk.ParameterObject.New()
default_rigid_parameter_map = parameter_object.GetDefaultParameterMap('rigid')
parameter_object.AddParameterMap(default_rigid_parameter_map)

def register_images(fixed_image, moving_image):
    fixed_image_import = itk.imread(fixed_image, itk.F)
    moving_image_import = itk.imread(moving_image, itk.F)

    # Load Elastix Image Filter Object
    elastix_object = itk.ElastixRegistrationMethod.New(fixed_image_import, moving_image_import)
    elastix_object.SetParameterObject(parameter_object)

    # Set additional options
    elastix_object.SetLogToConsole(True)
    elastix_object.LogToConsoleOn()

    # Update filter object (required)
    elastix_object.UpdateLargestPossibleRegion()

    # Results of Registration
    result_image = elastix_object.GetOutput()
    result_transform_parameters = elastix_object.GetTransformParameterObject()
    
    return result_image

In [14]:
list_pts = os.listdir(in_dir)
progress_bar = tqdm.tqdm(total=len(list_pts))

for pt in list_pts:
    in_pt = in_dir + '\\' + pt
    out_pt = out_dir + '\\' + pt
    
    # First copy everything over
    # Then register the images
    # Then delete the unregistered images

    shutil.copytree(in_pt, out_pt)

    print(f'Patient: {pt}')

    for seq in ['T2_ax', 'T1c_ax']:
        in_file = out_pt + "\\"  + seq + ".nii"
        in_file_aligned = out_pt + "\\"  + seq + "_aligned.nii"
        t1_file = out_pt + "\\"  + "T1_ax.nii"

        if os.path.isfile(in_file):
            print(f'Registering T1_ax with {seq}:', end=' ')
            out_nii_file = register_images(t1_file, in_file)
            if out_nii_file: print("Success!")
            itk.imwrite(out_nii_file, in_file_aligned)

            os.remove(in_file)
    
    progress_bar.update(1)

progress_bar.close()

  0%|          | 0/2 [00:10<?, ?it/s]


Patient: MR_102_anon
Registering T1_ax with T2_ax: Success!
Registering T1_ax with T1c_ax: 

 50%|█████     | 1/2 [00:08<00:08,  8.58s/it]

Success!
Patient: MR_231_anon
Registering T1_ax with T2_ax: Success!
Registering T1_ax with T1c_ax: 

100%|██████████| 2/2 [00:18<00:00,  9.35s/it]

Success!
